In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
main_df = pd.read_csv('./Data/main.csv')

## 4. Расчет конверсии

Посчитать конверсию перехода студентов из одного модуля в другой на каждом курсе.

Конверсия -  отношение количества студентов, приступивших к выполнению домашнего задания в ТЕКУЩЕМ модуле (если дз в модуле несколько, то считать по первому дз в модуле), к количеству студентов, сдавших задание в предыдущем модуле (если дз в модуле несколько, то считать по последнему дз в модуле).

In [3]:
# найдем количество тех кто начал какое-либо ДЗ, т.е. начал модуль   (191, 4)

start_df = main_df[main_df['is_homework'] == True] 

start_df = \
    start_df.groupby(['course_title', 'module_number']).\
    agg({'module_title' : 'first', 'course_field' : 'first', 'student_id' :'nunique'}).reset_index()
# сгруппировали по курсу, по модулю и посчитали уникальных студентов (если есть несколько ДЗ в модуле)

start_df.rename(columns={'student_id' : 'student_start'}, inplace = True)

In [4]:
main_df['status'].unique()

array(['done', 'start', 'fail', 'wait'], dtype=object)

In [5]:
finish_df = main_df[main_df['is_homework'] == True]
finish_df = finish_df[['student_id', 'course_title', 'module_number', 'lesson_number', 'status']]

finish_df['all_done'] = finish_df.groupby(['course_title', 'student_id', 'module_number'])['status'].\
    transform(lambda x: ~x.isin(['start', 'fail', 'wait']))
# проверяем что нет статусов ['start', 'fail', 'wait'], т.е. ВСЕ ДЗ зачтены

In [6]:
finish_df = finish_df[finish_df['all_done'] == True]
finish_df = finish_df.groupby(['course_title', 'module_number'])['student_id'].nunique().reset_index()    # (190, 3)

finish_df.rename(columns={'student_id' : 'student_finish'}, inplace = True)

In [7]:
conv_df = start_df.merge(finish_df, how='outer', on=['course_title', 'module_number']) # (191, 4)

In [8]:
def foo(x):
    l = list(x[:-1])
    l.insert(0,np.nan)
    return l

In [9]:
students_number = conv_df.groupby('course_title')['student_finish'].transform(foo).tolist()# используем student_finish

conv_df['student_finish'] = students_number

In [10]:
conv_df['conv'] = conv_df['student_start'] / conv_df['student_finish']
# делим тех кто начал новый на тех кто закончил предыдущий

conv_df.fillna(1, inplace=True)
# для тех модулей у которых NaN конверсию считаем, т.к. для них нет данных за предыдущий модуль

###  Bar-chart, отражающий конверсию перехода студентов из одного модуля в другой на каждом курсе. 

In [11]:
import matplotlib 
colors = matplotlib.colors.get_named_colors_mapping() # словарь цветов
colors_list = list(colors.values()) # взяли значения для выбора цвета

In [15]:
fig, axs = plt.subplots(15, 1, figsize=(10,40))  
fig.subplots_adjust(hspace=0.75)

i=0
for name, group in conv_df.groupby('course_title'):
    axs[i].bar(group['module_number'], group['conv'], color = colors_list[i*3])
    axs[i].set_ylabel('Конверсия', fontsize = 8)
    axs[i].set_xlabel('Номер модуля')
    axs[i].set_title(name, fontsize=12)
    axs[i].axhline(1, ls='--', color='k', linewidth=0.5)
    axs[i].set_xticks(list(group['module_number']))
    #axs[i].tick_params(axis='x', rotation=45)
        
    i+=1

plt.show()

### Bar-chart, отражающий конверсию перехода студентов из одного модуля в другой на каждом курсе. 

In [16]:
conv_df['module_title'] = conv_df['module_number'].astype("string") + '_' + conv_df['module_title']

In [17]:
fig, axs = plt.subplots(15, 1, figsize=(5,90))  
fig.subplots_adjust(hspace=0.5)

i=0
j=0
for field, field_group in conv_df.groupby('course_field'): 
    j += 10
    for name, group in field_group.groupby('course_title'): 
        median_value = round(group['conv'].median(), 2)# мединное значение конверсии для курса
        axs[i].barh(group['module_title'], group['conv'], height=0.4, label=field, alpha=0.9, color = colors_list[j])
        axs[i].set_xlabel('Коэфициент конверсии', fontsize = 10)
        axs[i].set_title(name, fontsize=12)        
        axs[i].set_frame_on(False) 
        axs[i].axvline(median_value, ls='--', color='r', label=f"Median = {median_value}", linewidth=0.5)
        axs[i].legend()
        axs[i].legend(bbox_to_anchor=(1.5,1))
        
        i+=1

plt.show()